In [222]:
import sys
sys.path.append("V:/biomoni/") 
from biomoni import Experiment
from BacillusScripts.BacillusVariableFeedrate_copy import Bacillus_vf
from biomoni import Model
import scipy
import pandas as pd
import numpy as np

In [223]:
a = Experiment("V:/biomoni/BacillusData/Stamm185", "F5", endpoint="F_end")

b= Bacillus_vf()


In [224]:
y0 = b.create_y0(a)
c = b.create_controls(a)
print(c)

{'feed_%_to_mL': 0.0864, 'feed_on': datetime.datetime(2021, 12, 15, 10, 44), 'add_IPTG': datetime.datetime(2021, 12, 15, 9, 3), 'feed_rate_mL_to_g': 1.14, 'csf': 336.59095416979716, 'cNH3': 184.61, 'gas_flow_lpm': 0.5, 'T': 37, 'gas_flow': 30.0, 'pressure': 1.0275395886889758, 'feedrate_glc': <scipy.interpolate.interpolate.interp1d object at 0x0000020F14F4DD10>, 'feedrate_volume': <scipy.interpolate.interpolate.interp1d object at 0x0000020F15821F40>, 'Fout': <scipy.interpolate.interpolate.interp1d object at 0x0000020F14F4D9A0>}


In [225]:


t_grid = a.dataset["off"].index.values
Carb_RR = []
V = 0.5
Mw_bm = 24.445          #g/mol molecular weight of biomass assuming composition of C=1,H=1.594,N=0.293,O=0.387,P=0.012,S=0.005
Mw_gluc = 180.156       #g/mol molecular weight of glucose C=6,H=12,O=6
Mw_RF = 376.36          #g/mol molecular weight of Riboflavin C=17,H=20,N=4,O=6

print(c["csf"])
F_C_in = (c["feedrate_glc"](t_grid))
print(F_C_in)
df_F_C_in = pd.DataFrame(data=F_C_in,index=t_grid, columns=["feedrate pure glucose g/h"])

F_C_in_cum = []

for t in t_grid:
    func = c["feedrate_glc"]
    cummulation = scipy.integrate.quad(func,0,t,limit=200)
    
    F_C_in_cum.append(cummulation)

cummulation_extr = [x[0] for x in F_C_in_cum]
df_F_C_in["Glc g_cum"] = cummulation_extr

#
# for t in t_grid:
#     cummulation = scipy.integrate.trapezoid(df_F_C_in[0],x =df_F_C_in.index,axis = 0)
#     F_C_in_cum.append(cummulation)


print(df_F_C_in)


amountC_given = (y0[0]/Mw_bm)+(y0[1]/Mw_gluc *6)+(y0[2]/Mw_RF *17)

# for t in t_grid:
    
#     nC_S = (float(a.dataset["off"]["Glucose [g/L]"].loc[[t]])*V)/Mw_gluc * 6
#     nC_X = (float(a.dataset["off"]["CDW_calc"].loc[[t]])*V)/ Mw_bm
#     nC_P = (float(a.dataset["off"]["RF [mg/L]"].loc[[t]])*V /1000)/ Mw_RF * 17
    
#     amountC_found = nC_S + nC_X + nC_P
#    # amountC_found = a.dataset["off"]["Glucose [g/L]"].loc[[t]]+a.dataset["off"]["CDW_calc"].loc[[t]]+a.dataset["off"]["RF [mg/L]"].loc[[t]]
   
#     Carb_RR.append(amountC_found)
    
    
    
    
#g = a.dataset["off"]["Glucose [g/L]"].loc[[0.0694444]]
f= a.dataset["off"]["Glucose [g/L]"]
#print(t_grid)
#print(Carb_RR)

#print(f)

336.59095416979716
[0.         0.         0.         3.29783726 3.61191678 3.85620141
 4.0830369  4.36221838 6.63057184 6.63057184 6.63057184 6.63057184
 6.63057184 6.63057184 6.63057184 3.14079738]


C:\Users\walkyre\AppData\Local\Temp/ipykernel_6636/637452770.py:17: IntegrationWarning:

The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.

C:\Users\walkyre\AppData\Local\Temp/ipykernel_6636/637452770.py:17: IntegrationWarning:

The maximum number of subdivisions (200) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.



           feedrate pure glucose g/h   Glc g_cum
0.069444                    0.000000    0.000000
16.069444                   0.000000    0.000000
17.752778                   0.000000    0.000000
18.586111                   3.297837    3.024035
19.836111                   3.611917    7.351651
20.802778                   3.856201   10.964868
21.702778                   4.083037   14.536210
22.852778                   4.362218   19.393836
39.452778                   6.630572  119.197885
40.719444                   6.630572  127.597166
41.952778                   6.630572  135.775234
43.402778                   6.630572  145.483693
44.736111                   6.630572  154.229783
46.052778                   6.630572  162.960357
47.336111                   6.630572  170.567817
63.719444                   3.140797  245.033447


In [226]:
import plotly.express as px


fig = px.scatter(x=df_F_C_in.index, y=df_F_C_in["feedrate pure glucose g/h"])
fig.show

<bound method BaseFigure.show of Figure({
    'data': [{'hovertemplate': 'x=%{x}<br>y=%{y}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': '#636efa', 'symbol': 'circle'},
              'mode': 'markers',
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'type': 'scatter',
              'x': array([ 0.06944444, 16.06944444, 17.75277778, 18.58611111, 19.83611111,
                          20.80277778, 21.70277778, 22.85277778, 39.45277778, 40.71944444,
                          41.95277778, 43.40277778, 44.73611111, 46.05277778, 47.33611111,
                          63.71944444]),
              'xaxis': 'x',
              'y': array([0.        , 0.        , 0.        , 3.29783726, 3.61191678, 3.85620141,
                          4.0830369 , 4.36221838, 6.63057184, 6.63057184, 6.63057184, 6.63057184,
                          6.63057184, 6.63057184, 6.63057184, 3.14079738]),
              'yax

In [ ]:
def calc_CRR(self,experiment, V, y0):

    Mw_bm = 24.445          #g/mol molecular weight of biomass assuming composition of C=1,H=1.594,N=0.293,O=0.387,P=0.012,S=0.005
    Mw_gluc = 180.156       #g/mol molecular weight of glucose C=6,H=12,O=6
    Mw_RF = 376.36          #g/mol molecular weight of Riboflavin C=17,H=20,N=4,O=6
    F_C_in = c["feedrate_glc"](t_grid)
        
    amountC_given = (y0[0]/Mw_bm)+(y0[1]/Mw_gluc *6)+(y0[2]/Mw_RF *17)

    t_grid = experiment.dataset["off"].index.values
    Carb_RR=[]
    for t in t_grid:
        
        nC_S = float(a.dataset["off"]["Glucose [g/L]"].loc[[t]])*V
        nC_X = float(a.dataset["off"]["CDW_calc"].loc[[t]])*V
        nC_P = float(a.dataset["off"]["RF [mg/L]"].loc[[t]])* V /1000
        
        amountC_found = nC_S + nC_X + nC_P+ nC_CO2_cum
        Carb_RR = amountC_found / amountC_given
        
        return Carb_RR